In [1]:
import pandas as pd
import numpy as np
import geopandas as gp

In [27]:
taz_data=gp.read_file('../../scenarios/convert/SF-CHAMP Outputs/TAZ2454_clean.shp')
taz_data.head()

,TAZ,SUPERD,TAZ1454,AREALAND,AREAWATR,WATRACRE,SD,COUNTY,SFBLKGRP,X_COORD,Y_COORD,Nhood,nhood_num,lg_nhood,lg_nhd_num,SQ_MILE,DIST20,geometry
0,2005,19,1005,73176114,0,0.0,19,4,0,6.062583e+06,2.142762e+06,East Bay,50,East Bay,14,2.62483,15,"POLYGON ((6060457.736 2148569.630, 6060773.118..."
1,1999,19,999,7099208,0,0.0,19,4,0,6.056350e+06,2.135772e+06,East Bay,50,East Bay,14,0.25465,15,"POLYGON ((6058324.039 2136300.032, 6058221.296..."
2,1998,19,998,11566088,0,0.0,19,4,0,6.054933e+06,2.133579e+06,East Bay,50,East Bay,14,0.41488,15,"POLYGON ((6052051.841 2132957.805, 6052222.062..."
3,2000,19,1000,7491442,0,0.0,19,4,0,6.054171e+06,2.135816e+06,East Bay,50,East Bay,14,0.26872,15,"POLYGON ((6055539.212 2137189.697, 6055534.438..."
4,2001,19,1001,6203025,0,0.0,19,4,0,6.051967e+06,2.136249e+06,East Bay,50,East Bay,14,0.22250,15,"POLYGON ((6051856.588 2134855.280, 6051567.202..."


In [28]:
input_plans=pd.read_csv('../output/SF_TourPurposeChoiceModel/sf-tscore-all-trips-mc-calibration-trial-3__bii/plans.csv.gz')
output_plans=pd.read_csv('../output/SF_TourPurposeChoiceModel/sf-tscore-all-trips-mc-calibration-trial-3__bii/ITERS/it.10/10.plans.csv.gz')
agent_attribs=pd.read_csv('../test/input/sf-test-scenario/Inputs/IX-Trips-Updated/agent_attribs.csv.gz')

In [42]:
def county(x):
    if x<1191:
        return "San Francisco"
    elif x<1347:
        return "San Mateo"
    elif x<1715:
        return "Santa Clara"
    elif x<2040:
        return "Almeda"
    elif x<2211:
        return "Contra Costa"
    elif x<2291:
        return "Solano"
    elif x<2318:
        return "Napa"
    elif x<2404:
        return 'Sonoma'
    else:
        return "Marin"

def taz_cleanup(taz_df):
    
    taz_df['County']=taz_df['TAZ'].apply(county)
    taz_df['District']=np.where(taz_df['County']=='San Francisco',
                               taz_df['DIST20'],
                               taz_df['County'])
    districts=taz_df[['District','geometry']].dissolve('District')
    
    districts=districts.to_crs('EPSG:26910')
    
    districts=districts.reset_index()
    return districts

def plans_cleanup(plans,districts,agent_attribs):
    
    plans=plans[plans['planSelected']==True]
    plans['Mode']=plans['legMode'].shift(-1)
    plans['maxPlanElementIndex']=plans.groupby('personId')['planElementIndex'].transform(max)
    plans=plans[plans['activityType'].notnull()]
    
    plans['origin_activityType']=plans['activityType']
    plans['tourPurpose']=plans['activityType'].shift(-1)
    
    plans['geometry']=gp.points_from_xy(plans['activityLocationX'],plans['activityLocationY'],crs='EPSG:26910')
    
    plans_gdf=gp.GeoDataFrame(plans,geometry='geometry')
    
    plans_district=gp.sjoin(plans_gdf,districts,how='left',predicate='within')
    
    plans_district['origin_district']=plans_district['District']
    plans_district['destination_district']=plans_district['origin_district'].shift(-1)
    
    plans_district=plans_district[plans_district['planElementIndex']!=plans_district['maxPlanElementIndex']]
    
    plans_df=pd.merge(plans_district,agent_attribs,left_on=['personId'],right_on=['person_id'],how='left')
    
#     plans_district['startTime']=plans_district['activityEndTime']
#     plans_district['startTime']=plans_district['activityEndTime']

    return plans_df[['personId','hhno',  'pgend', 'pagey', 'hhincome', 'hhsize','vehicles',
                     'planIndex', 'planScore', 'planSelected', 'planElementType',
                     'planElementIndex', 'Mode',
                     'maxPlanElementIndex', 'origin_activityType', 'tourPurpose', 'geometry',
                     'index_right', 'District', 'origin_district', 'destination_district']]
    

In [32]:
%%time
districts=taz_cleanup(taz_data)

districts.head()

CPU times: total: 2.25 s
Wall time: 2.25 s


,District,geometry
0,1,"POLYGON ((551128.993 4181109.809, 551124.990 4..."
1,2,"MULTIPOLYGON (((551079.065 4180348.942, 551080..."
2,3,"MULTIPOLYGON (((551006.693 4182737.702, 551021..."
3,4,"POLYGON ((549129.160 4179082.059, 549127.183 4..."
4,5,"POLYGON ((550654.516 4178259.039, 550647.268 4..."


In [43]:
%%time
input_plans_cleaned=plans_cleanup(input_plans,districts,agent_attribs)
output_plans_cleaned=plans_cleanup(output_plans,districts,agent_attribs)

C:\Users\jawad\AppData\Local\Temp\ipykernel_13528\1549724210.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plans['Mode']=plans['legMode'].shift(-1)
C:\Users\jawad\AppData\Local\Temp\ipykernel_13528\1549724210.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plans['maxPlanElementIndex']=plans.groupby('personId')['planElementIndex'].transform(max)


CPU times: total: 1min 6s
Wall time: 1min 6s


In [61]:
mc_params=pd.read_csv('../test/input/sf-test-scenario/Inputs/ActivitySimMC-TSCORE-trial-3.csv')
mc_params.head()

,model,tourType,variable,alternative,units,latentClass,value
0,modeChoice,Person,cost,walk_transit,util/min,work,-1.0
1,modeChoice,Person,cost,walk,util/min,work,-1.0
2,modeChoice,Person,cost,ride_hail,util/min,work,-1.0
3,modeChoice,Person,cost,hov3_teleportation,util/min,work,-1.0
4,modeChoice,Person,cost,hov3,util/min,work,-1.0


In [63]:
mc_params.variable.unique()

array(['cost', 'ascNoAuto', 'ascAutoSufficient', 'ascAutoDeficient',
       'age1619', 'age010', 'waitTime', 'vehicleTime', 'transfer',
       'shortWalkDist', 'shortBikeDist', 'originFarFromTransit',
       'originCloseToTransit', 'longWalkDist', 'longBikeDist',
       'egressTime', 'destFarFromTransit', 'destCloseToTransit',
       'originZDI', 'destZDI', 'CBD', 'walkTime', 'age16P', 'hhSize1',
       'hhSize2', 'shortDrive'], dtype=object)

In [64]:
df1=input_plans_cleaned.groupby(['tourPurpose','vehicles','Mode']).agg({'Mode':'count'}).rename(columns={'Mode':'SF CHAMP Count'}).reset_index()

output_plans_cleaned['Mode']=np.where(output_plans_cleaned['Mode'].isnull(),
                                     'Not Assigned',
                                     output_plans_cleaned['Mode'])
df2=output_plans_cleaned.groupby(['tourPurpose','vehicles','Mode']).agg({'Mode':'count'}).rename(columns={'Mode':'BEAM Count'}).reset_index()

df3=pd.merge(df1,df2,on=['tourPurpose','vehicles','Mode'],how='outer')

df3['Change_in_ASC']=np.log(df3['SF CHAMP Count']/df3['BEAM Count'])

df3.head(10)
#.to_csv('Temp/mode_counts.csv')

,tourPurpose,vehicles,Mode,SF CHAMP Count,BEAM Count,Change_in_ASC
0,eatout,auto_deficient,bike,79.0,16.0,1.596859
1,eatout,auto_deficient,car,1041.0,125.0,2.119623
2,eatout,auto_deficient,hov2,439.0,85.0,1.641848
3,eatout,auto_deficient,hov2_teleportation,199.0,57.0,1.250254
4,eatout,auto_deficient,hov3,169.0,34.0,1.603538
5,eatout,auto_deficient,hov3_teleportation,174.0,57.0,1.116004
6,eatout,auto_deficient,ride_hail,235.0,29.0,2.092290
7,eatout,auto_deficient,walk,1227.0,2053.0,-0.514730
8,eatout,auto_deficient,walk_transit,305.0,238.0,0.248041
9,eatout,auto_sufficient,bike,51.0,10.0,1.629241


In [65]:
df3.vehicles.unique()

array(['auto_deficient', 'auto_sufficient', 'no_auto'], dtype=object)

In [67]:
def asc_var(x):
    
    if x=='auto_deficient':
        return 'ascNoAuto'
    elif x=='auto_sufficient':
        return 'ascAutoSufficient'
    else:
        return 'ascAutoDeficient'
    
df3['variable']=df3['vehicles'].apply(asc_var)
df3.head()

,tourPurpose,vehicles,Mode,SF CHAMP Count,BEAM Count,Change_in_ASC,variable
0,eatout,auto_deficient,bike,79.0,16.0,1.596859,ascNoAuto
1,eatout,auto_deficient,car,1041.0,125.0,2.119623,ascNoAuto
2,eatout,auto_deficient,hov2,439.0,85.0,1.641848,ascNoAuto
3,eatout,auto_deficient,hov2_teleportation,199.0,57.0,1.250254,ascNoAuto
4,eatout,auto_deficient,hov3,169.0,34.0,1.603538,ascNoAuto


In [68]:
mc_params.head(10)

,model,tourType,variable,alternative,units,latentClass,value
0,modeChoice,Person,cost,walk_transit,util/min,work,-1.0
1,modeChoice,Person,cost,walk,util/min,work,-1.0
2,modeChoice,Person,cost,ride_hail,util/min,work,-1.0
3,modeChoice,Person,cost,hov3_teleportation,util/min,work,-1.0
4,modeChoice,Person,cost,hov3,util/min,work,-1.0
5,modeChoice,Person,cost,hov2_teleportation,util/min,work,-1.0
6,modeChoice,Person,cost,hov2,util/min,work,-1.0
7,modeChoice,Person,cost,drive_transit,util/min,work,-1.0
8,modeChoice,Person,cost,car,util/min,work,-1.0
9,modeChoice,Person,cost,bike,util/min,work,-1.0


In [78]:
updated_mc_param=pd.merge(mc_params,df3,
                          left_on=['alternative','latentClass','variable'],
                         right_on=['Mode','tourPurpose','variable'],
                         how='left')
updated_mc_param['Change_in_ASC']=updated_mc_param['Change_in_ASC'].fillna(0)

updated_mc_param['updated_value']=updated_mc_param['value']+updated_mc_param['Change_in_ASC']

#updated_mc_param.drop(columns=['Mode','tourPurpose','Change_in_ASC','value'],inplace=True)

#updated_mc_param=updated_mc_param.rename(columns={'updated_value':'value'})

updated_mc_param.to_csv('../test/input/sf-test-scenario/Inputs/MC_Parameters_Trial_4_Detailed.csv',index=False)

In [80]:
df3.columns

Index(['tourPurpose', 'vehicles', 'Mode', 'SF CHAMP Count', 'BEAM Count',
       'Change_in_ASC', 'variable'],
      dtype='object')

In [81]:
updated_mc_param.drop(columns=['tourPurpose', 'vehicles', 'Mode', 'SF CHAMP Count', 'BEAM Count',
       'Change_in_ASC','value'],inplace=True)

updated_mc_param=updated_mc_param.rename(columns={'updated_value':'value'})

updated_mc_param.head()

,model,tourType,variable,alternative,units,latentClass,value
0,modeChoice,Person,cost,walk_transit,util/min,work,-1.0
1,modeChoice,Person,cost,walk,util/min,work,-1.0
2,modeChoice,Person,cost,ride_hail,util/min,work,-1.0
3,modeChoice,Person,cost,hov3_teleportation,util/min,work,-1.0
4,modeChoice,Person,cost,hov3,util/min,work,-1.0


In [82]:
updated_mc_param.to_csv('../test/input/sf-test-scenario/Inputs/ActivitySimMC-TSCORE-trial-4.csv',index=False)

In [56]:
output_plans_cleaned.head(10)

,personId,hhno,pgend,pagey,hhincome,hhsize,vehicles,planIndex,planScore,planSelected,...,planElementIndex,Mode,maxPlanElementIndex,origin_activityType,tourPurpose,geometry,index_right,District,origin_district,destination_district
0,63-2,63,female,32,38016,8.0,auto_sufficient,4,379.596677,True,...,0,Not Assigned,8,home,othmaint,POINT (551662.442 4173738.730),8.0,9,9,10
1,63-2,63,female,32,38016,8.0,auto_sufficient,4,379.596677,True,...,2,Not Assigned,8,othmaint,home,POINT (549567.289 4175127.022),9.0,10,10,9
2,63-2,63,female,32,38016,8.0,auto_sufficient,4,379.596677,True,...,4,Not Assigned,8,home,othmaint,POINT (551662.442 4173738.730),8.0,9,9,9
3,63-2,63,female,32,38016,8.0,auto_sufficient,4,379.596677,True,...,6,Not Assigned,8,othmaint,home,POINT (552229.027 4175727.623),8.0,9,9,9
4,63-3,63,female,30,38016,8.0,auto_sufficient,2,930.963822,True,...,0,Not Assigned,16,home,escort,POINT (551662.442 4173738.730),8.0,9,9,10
5,63-3,63,female,30,38016,8.0,auto_sufficient,2,930.963822,True,...,2,Not Assigned,16,escort,home,POINT (548946.307 4174811.777),9.0,10,10,9
6,63-3,63,female,30,38016,8.0,auto_sufficient,2,930.963822,True,...,4,Not Assigned,16,home,escort,POINT (551662.442 4173738.730),8.0,9,9,10
7,63-3,63,female,30,38016,8.0,auto_sufficient,2,930.963822,True,...,6,Not Assigned,16,escort,home,POINT (549880.859 4174499.154),9.0,10,10,9
8,63-3,63,female,30,38016,8.0,auto_sufficient,2,930.963822,True,...,8,Not Assigned,16,home,shopping,POINT (551662.442 4173738.730),8.0,9,9,1
9,63-3,63,female,30,38016,8.0,auto_sufficient,2,930.963822,True,...,10,Not Assigned,16,shopping,home,POINT (553319.371 4183077.012),0.0,1,1,9


In [57]:
trips_comparison=pd.merge(input_plans_cleaned[['personId', 'pgend', 'pagey', 'hhincome', 'hhsize','vehicles',
                                               'planElementIndex', 'Mode', 'origin_activityType', 'tourPurpose',
                                               'origin_district', 'destination_district']],
                         output_plans_cleaned[['personId', 'pgend', 'pagey', 'hhincome', 'hhsize','vehicles',
                                               'planElementIndex', 'Mode', 'origin_activityType', 'tourPurpose',
                                               'origin_district', 'destination_district']],
                         on=['personId', 'pgend', 'pagey', 'hhincome', 'hhsize','vehicles','planElementIndex'],
                         how='inner',
                         suffixes=('_SFCHAMP','_BEAM'))

trips_comparison.head()

,personId,pgend,pagey,hhincome,hhsize,vehicles,planElementIndex,Mode_SFCHAMP,origin_activityType_SFCHAMP,tourPurpose_SFCHAMP,origin_district_SFCHAMP,destination_district_SFCHAMP,Mode_BEAM,origin_activityType_BEAM,tourPurpose_BEAM,origin_district_BEAM,destination_district_BEAM
0,63-2,female,32,38016,8.0,auto_sufficient,0,hov3_teleportation,home,othmaint,9,10,Not Assigned,home,othmaint,9,10
1,63-2,female,32,38016,8.0,auto_sufficient,2,ride_hail,othmaint,home,10,9,Not Assigned,othmaint,home,10,9
2,63-2,female,32,38016,8.0,auto_sufficient,4,walk,home,othmaint,9,9,Not Assigned,home,othmaint,9,9
3,63-2,female,32,38016,8.0,auto_sufficient,6,walk,othmaint,home,9,9,Not Assigned,othmaint,home,9,9
4,63-3,female,30,38016,8.0,auto_sufficient,0,hov3_teleportation,home,escort,9,10,Not Assigned,home,escort,9,10


In [58]:
def replanning(df):
    if df['Mode_BEAM']=='Not Assigned':
        return 'Null Mode'
    
    elif df['Mode_BEAM']==df['Mode_SFCHAMP']:
        return 'Mode not replanned'
    
    else:
        return 'Mode Replanned'
    
trips_comparison['Mode_Replanning']=trips_comparison.apply(replanning,axis=1)

In [71]:
trips_comparison.Mode_Replanning.value_counts(normalize=True)

Mode Replanned        0.478929
Null Mode             0.280839
Mode not replanned    0.240232
Name: Mode_Replanning, dtype: float64

In [55]:
output_plans_cleaned['Mode'].value_counts(dropna=False)

walk                  74029
Not Assigned          40445
walk_transit           8881
car                    5886
hov2_teleportation     4943
hov2                   3479
hov3_teleportation     3038
hov3                   1141
bike_transit            900
bike                    640
ride_hail               538
ride_hail_pooled         66
other                    29
Name: Mode, dtype: int64

In [45]:
input_plans_cleaned[input_plans_cleaned['index_right'].isnull()]

,personId,hhno,pgend,pagey,hhincome,hhsize,vehicles,planIndex,planScore,planSelected,...,planElementIndex,Mode,maxPlanElementIndex,origin_activityType,tourPurpose,geometry,index_right,District,origin_district,destination_district
3210,ix-8134-3,8134,male,36,87015,6.0,auto_deficient,0,0.0,True,...,2,car,4,work,home,POINT (553795.225 4168899.055),NaN,NaN,NaN,9
3550,8953-2,8953,male,53,36262,3.0,auto_sufficient,0,0.0,True,...,2,walk,8,work,eatout,POINT (545251.626 4173848.876),NaN,NaN,NaN,12
3552,8953-2,8953,male,53,36262,3.0,auto_sufficient,0,0.0,True,...,6,hov2,8,work,home,POINT (545251.626 4173848.876),NaN,NaN,NaN,10
4464,ix-10893-1,10893,female,60,58353,2.0,auto_deficient,0,0.0,True,...,6,car,8,shopping,home,POINT (560723.671 4186039.688),NaN,NaN,NaN,9
5325,ix-12657-1,12657,male,60,3293,2.0,auto_sufficient,0,0.0,True,...,2,car,4,work,home,POINT (553795.225 4168899.055),NaN,NaN,NaN,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140779,ix-2734940-3,2734940,male,27,34323,3.0,auto_deficient,0,0.0,True,...,0,car,4,work,work,POINT (544246.946 4191286.758),NaN,NaN,NaN,San Mateo
141280,ix-2740930-2,2740930,male,37,49262,4.0,auto_deficient,0,0.0,True,...,0,car,4,work,work,POINT (544246.946 4191286.758),NaN,NaN,NaN,12
141458,ix-2743908-1,2743908,female,81,30828,1.0,auto_sufficient,0,0.0,True,...,0,car,4,home,shopping,POINT (530481.606 4195035.742),NaN,NaN,NaN,8
141460,ix-2743956-1,2743956,female,64,68483,1.0,auto_sufficient,0,0.0,True,...,0,car,4,home,work,POINT (530481.606 4195035.742),NaN,NaN,NaN,9
